In [1]:
!pip install git+https://github.com/sebastian-sz/efficientnet-v2-keras@main

  Running command git clone --filter=blob:none --quiet https://github.com/sebastian-sz/efficientnet-v2-keras 'C:\Users\poscouser\AppData\Local\Temp\pip-req-build-o6nv4v14'


  Cloning https://github.com/sebastian-sz/efficientnet-v2-keras (to revision main) to c:\users\poscouser\appdata\local\temp\pip-req-build-o6nv4v14
  Resolved https://github.com/sebastian-sz/efficientnet-v2-keras to commit c7aea25794d04a6edfec22241e3b3e2cc3547f3e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet-v2-keras: filename=efficientnet_v2_keras-1.0-py3-none-any.whl size=16193 sha256=d5b7ebd950e24760e25d2cabb88b11ca17cd805469f39e9507f2b55076bab101
  Stored in directory: C:\Users\poscouser\AppData\Local\Temp\pip-ephem-wheel-cache-4b8d0dsw\wheels\cc\58\f2\32979a5ef3177be425bcb47625702a823e3656398798ec577a
Successfully built efficientnet-v2-keras


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from efficientnet_v2 import EfficientNetV2M

%matplotlib inline

In [3]:
class_num = 12
batch_size = 32
target_size=(200,200)

gen = ImageDataGenerator(rescale=1/255.)
train_gen = gen.flow_from_directory('./data/train/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)
valid_gen = gen.flow_from_directory('./data/valid/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)
test_gen = gen.flow_from_directory('./data/test/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)

Found 36720 images belonging to 12 classes.
Found 12240 images belonging to 12 classes.
Found 12240 images belonging to 12 classes.


In [4]:
transfer_base = EfficientNetV2M(weights='imagenet',
                          include_top=False,        # class 3개로 분류 학습시켜야 해서 레이어 분리시킨 후, Desnse(3) 추가..?
                          input_shape=(200, 200, 3))
transfer_base.summary()

Model: "efficientnetv2-m"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 100, 100, 24) 648         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 100, 100, 24) 96          stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 100, 100, 24) 0           stem_bn[0][0]                    
___________________________________________________________________________________

In [5]:
model = models.Sequential()
model.add(transfer_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(class_num, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetv2-m (Functional (None, 7, 7, 1280)        53150388  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dense_1 (Dense)              (None, 12)                3084      
Total params: 53,481,408
Trainable params: 53,189,376
Non-trainable params: 292,032
_________________________________________________________________


In [6]:
#저장할 모델명(폴더명)
model_name = 'EffNetV2M_12class'

#폴더생성
if 'save_models' not in os.listdir(): os.mkdir('save_models')
if model_name not in os.listdir('save_models'): os.mkdir('./save_models/{}'.format(model_name))
#모델 저장 path
path_checkpoint = "./save_models/{}/".format(model_name) 
#모델 저장 코드(weight만 저장)
es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5, restore_best_weights=True)
modelckpt_callback = tf.keras.callbacks.ModelCheckpoint(monitor="val_loss", filepath=path_checkpoint, verbose=1, save_weights_only=False, save_best_only=True, save_format='tf')

#fit
optimizer = tf.keras.optimizers.Adam(lr=0.00001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = 'accuracy')
history = model.fit(train_gen, validation_data=valid_gen, batch_size=batch_size, epochs=5, callbacks=[es_callback, modelckpt_callback])

Epoch 1/10
1148/1148 [==============================] - ETA: 0s - loss: 0.9155 - accuracy: 0.7642
Epoch 00001: val_loss improved from inf to 0.28509, saving model to ./save_models/EffNetV2M_12class/
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./save_models/EffNetV2M_12class/assets
1148/1148 [==============================] - 702s 612ms/step - loss: 0.9155 - accuracy: 0.7642 - val_loss: 0.2851 - val_accuracy: 0.9157
Epoch 2/10
1148/1148 [==============================] - ETA: 0s - loss: 0.0956 - accuracy: 0.9764
Epoch 00002: val_loss improved from 0.28509 to 0.25118, saving model to ./save_models/EffNetV2M_12class/
INFO:tensorflow:Assets written to: ./save_models/EffNetV2M_12class/assets
1148/1148 [==============================] - 682s 594ms/step - loss: 0.0956 - a


KeyboardInterrupt



In [7]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('ACCURACY')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.grid()
plt.legend(['train','validation'], loc = 'best')

NameError: name 'history' is not defined

In [8]:
#저장된 최고 성능 모델 불러오기
model_loaded = tf.keras.models.load_model(path_checkpoint)

#test score 확인
score = model_loaded.evaluate(test_gen) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

383/383 [==============================] - 47s 121ms/step - loss: 0.2086 - accuracy: 0.9482
Test loss: 0.20861004292964935
Test accuracy: 0.9482026100158691
